In [1]:
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import Slider, ColumnDataSource, CustomJS

output_notebook()

Loading BokehJS ...

A continuous time sinewave whose frequency is $f_0$ Hz (cycles per second) is entirely and unambiguously defined by the following function of time, $t$ seconds...

$x(t)=\sin(2\pi f_ot)$

* $f_ot=\frac{cycles}{second}\times seconds=cycles$ ...so it's a measure of how many cycles (and partial cycles) have occured by the given time, $t$.
* multiplying by $2\pi$ converts the number of cycles to an angle in radians (1 cycle = $2\pi$ radians)

You can see this graphically below: [[1]](#foot1)<a name="foot1-back"></a>

In [9]:
t = np.arange(0.0, 2.0, 0.01)
f_0 = 1

plotData = ColumnDataSource(data=dict(x=t, y=np.sin(2*np.pi*f_0*t)))

plot=figure(title='"Continuous" sinewave', plot_width=500, plot_height=500, tools=[], x_axis_label="time, t (s)")
plot.line('x', 'y', legend="sin(2\u03C0f\u2080t)", source=plotData)

slider=Slider(start=0, end=5, value=f_0, step=0.01, title="f\u2080 (Hz)")

updateData = CustomJS(args=dict(source=plotData, slider=slider), code="""
    var f_0=slider.value;
    
    var data = source.data;
    var x=data['x'];
    var y=data['y'];
    
    for(var i=0; i < x.length; i++) {
        y[i] = Math.sin(2*Math.PI*f_0*x[i]);
    }
    
    source.change.emit();
""")
slider.js_on_change('value', updateData)

show(row(plot, slider))

However, if we sample the sinewave at a rate of $f_s$ samples per second (so the time between each sample is $t_s=\frac{1}{f_s}$) then instead of a continuous function of time we get an indexed series of discrete values like...

$x[n]=\sin(2\pi f_ont_s)$

So it's basically the same calculation but instead of continuous time, $t$ we're measuring the number of cycles at discrete points in time, $n \times t_s$. ($n$ is an integer.)

Use the sliders below to explore the effect of different sample frequencies on different sinewave frequencies...

In [14]:
f_0 = 1
f_s = 20
t_s = 1/f_s

n = np.arange(f_s*2) # plot 2 seconds worth of cycles

plotData = ColumnDataSource(data=dict(x=n*t_s, y0=np.zeros(len(n)), y1=np.sin(2*np.pi*f_0*n*t_s)))

updateData = CustomJS(args=dict(source=plotData), code="""
    var f_0=f_0.value;
    var f_s=f_s.value;
    var t_s=1/f_s;
    var data = source.data;
    
    var x  = data['x']=[];
    var y0 = data['y0']=[];
    var y1 = data['y1']=[];
    
    for(var i=0; i < f_s*2; i++) {
        x.push(i*t_s);
        y0.push(0);
        y1.push(Math.sin(2*Math.PI*f_0*x[i]));
    }
    
    source.change.emit();
""")

slider_f0=Slider(start=0, end=5, value=f_0, step=0.01, title="f\u2080 (Hz)")
slider_f0.js_on_change('value', updateData)
updateData.args['f_0']=slider_f0
slider_fs=Slider(start=0, end=50, value=f_s, step=0.01, title="f\u209B (samples/sec)")
slider_fs.js_on_change('value', updateData)
updateData.args['f_s']=slider_fs

plot=figure(title="Discrete sinewave, f\u209B samples/second", plot_width=800, plot_height=500, tools=[])
plot.segment('x', 'y0', 'x', 'y1', source=plotData)
plot.circle('x','y1', source=plotData)

show(column(plot, row(slider_f0,slider_fs)))

The trouble is, that series of disctrete sampled values no longer unambigiously defines the original sinewave. Whatever combination of sinewave frequency and sampling frequency we use, there will always be other sinewaves with different frequencies that would give exactly the same series of values when sampled at the same rate. In fact, as we'll see later, there are an infinite number of such sinewaves!

The chart below shows two frequencies that produce the same set of samples for the given sinewave frequency and sample frequency. The red dotted line shows the sinewave at the frequency, $f_0$, chosen by the slider. The green line shows another sinewave at a different frequency producing the same set of sampled values using the same sample frequency.

In [7]:
f_0 = 1
f_s = 10
t_s = 1/f_s

n = np.arange(f_s*2) # plot 2 seconds worth of cycles
t = np.arange(0.0, 2.0, 0.001)

plotData = ColumnDataSource(data=dict(x=n*t_s, y0=np.zeros(len(n)), y1=np.sin(2*np.pi*f_0*n*t_s)))
aliasPlotData = ColumnDataSource(data=dict(t=t, yAlias0=np.sin(2*np.pi*f_0*t),
                                                yAlias1=np.sin(2*np.pi*(f_0+f_s)*t)))

updateData = CustomJS(args=dict(source=plotData), code="""
    var f_0=f_0.value;
    var f_s=f_s.value;
    var t_s=1/f_s;
    var data = source.data;
    
    var x  = data['x']=[];
    var y0 = data['y0']=[];
    var y1 = data['y1']=[];
    
    for(var i=0; i < f_s*2; i++) {
        x.push(i*t_s);
        y0.push(0);
        y1.push(Math.sin(2*Math.PI*f_0*x[i]));
    }
    
    var aliasData = aliasSource.data;
    var t = aliasData['t'];
    var yAlias0 = aliasData['yAlias0'];
    var yAlias1 = aliasData['yAlias1'];
    
    for(var i=0; i < t.length; i++) {
        yAlias0[i] = Math.sin(2*Math.PI*f_0*t[i]);
        yAlias1[i] = Math.sin(2*Math.PI*(f_0+f_s)*t[i]);
    }
    
    source.change.emit();
""")
updateData.args['aliasSource']=aliasPlotData


slider_f0=Slider(start=0, end=5, value=f_0, step=0.01, title="f\u2080 (Hz)")
slider_f0.js_on_change('value', updateData)
updateData.args['f_0']=slider_f0
slider_fs=Slider(start=0, end=50, value=f_s, step=0.01, title="f\u209B (samples/sec)")
slider_fs.js_on_change('value', updateData)
updateData.args['f_s']=slider_fs

plot=figure(title="Ambigous discrete sinewaves", plot_width=800, plot_height=500, tools=[])
plot.segment('x', 'y0', 'x', 'y1', source=plotData, line_width=2)
plot.circle('x','y1', source=plotData)
plot.line('t', 'yAlias0', source=aliasPlotData, color='red', line_dash='4 4')
plot.line('t', 'yAlias1', source=aliasPlotData, color='green', alpha=0.5)


show(column(plot, row(slider_f0,slider_fs)))

Because the sinewave is periodic every $2\pi$, you can add any multiple of $2\pi$ to the angle to get the same values...

$\sin(2\pi f_0nt_s)=\sin(2\pi f_0nt_s+2\pi m) \quad\text{(where $m$ is any integer).}$

We can rearrange this to start investigating what frequencies produce identical sampled values...

$\sin(2\pi f_0nt_s+2\pi m)=\sin(2\pi(f_0nt_s+m))$

$=\sin(2\pi(f_0+\frac{m}{nt_s})nt_s)$

Remember that the sample frequency $f_s=\frac{1}{t_s}$ so we can write the above as...

$\sin(2\pi(f_0+\frac{m}{nt_s})nt_s)=\sin(2\pi(f_0+\frac{m}{n}\frac{1}{t_s})nt_s)$

$=\sin(2\pi(f_0+\frac{m}{n}f_s)nt_s)$

So a frequency of $f_0+\frac{m}{n}f_s$ will give the same set of samples as $f_0$ when sampled at the same rate $f_s$.

That $\frac{m}{n}$ multiplier can be a bit confusing. We said $m$ can be any integer and $n$ is the index integer of the samples.

If $\Delta f=\frac{m}{n}f_s$ then $m=n\frac{\Delta f}{f_s}$ and the only way for $m$ to be an integer for all $n$ is if $\frac{\Delta f}{f_s}$ is also an integer, $k$. $\frac{\Delta f}{f_s}=k$ so $\Delta f=kf_s$.

Which is a long-winded way of saying any frequencies of $f_0+kf_s$, where $k$ is any integer, will produce the same discrete values when sampled at $f_s$. In other words, there are an infinite number of sinewaves that will produce the same discrete values when sampled at the same rate!

---

**Footnotes:**

<a name="foot1"></a> [[1]](#foot1-back) Of course the code used to plot this graph doesn't really plot a continuous sinewave. I'm cheating by drawing a line between a lot of close together sampled points to give the illusion of a continuous curve. Please ignore that the purpose of the illustration!